In [2]:
import pandas_datareader.data as web
import datetime
import pandas as pd

# Scrap_label (stock price)

In [15]:
#設定時間(一周)
end_date = datetime.date(2020,10,22)#datetime.date.today() - datetime.timedelta(days=1)#每天自動於美國時間0點開始計算前七天的data
start_date = end_date - datetime.timedelta(days=8)
start_delta = end_date - datetime.timedelta(days=17) #多取一個禮拜的data以計算return

In [16]:
#設定抓取目標
company_symbol = ['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT']
company_symbol

['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT']

In [248]:
#抓取股價並存檔
for name in company_symbol:
    data = web.DataReader(name, 'stooq',start_delta, end_date)
    #drop unnecessary column & reverse index
    data = data.drop(["Open", "High", "Low", "Volume"], axis=1).reindex(index=data.index[::-1])
    data['daily_return'] = ((data['Close']/data['Close'].shift(1))-1) * 100 #算出return
    
    data.insert(loc=2, column='UpDown',value='')#insert new column
    data.loc[data['daily_return'] > 1, 'UpDown'] = 1#漲
    data.loc[data['daily_return'] < -1, 'UpDown'] = -1#跌
    data['UpDown'] = data['UpDown'].replace('', 0)#不漲不跌

    for i in range(data.size):#刪除超過一周的data
        if data.index[i] >= start_date:
            data = data[i:]
            break
    data['date'] = data.index#保留date，儲存成csv時index會並成0~n    
    
    data.to_csv(name + '_stock_price' + '.csv',index = False)

In [249]:
data

,Close,daily_return,UpDown,date
Date,,,,
2020-10-14,220.86,-0.897424,0,2020-10-14
2020-10-15,219.66,-0.543331,0,2020-10-15
2020-10-16,219.66,0.000000,0,2020-10-16
2020-10-19,214.22,-2.476555,-1,2020-10-19
2020-10-20,214.65,0.200728,0,2020-10-20
2020-10-21,214.80,0.069881,0,2020-10-21
2020-10-22,214.89,0.041899,0,2020-10-22


# Scrap_feature (tweets)

In [17]:
#設定API
consumer_key = 'Li266Kl7Wv8QgbQ2mNL3jBx0N'
consumer_secret = '8QCryShnQ5Ct9DFl1t8yUwssqwMJ4TSffOjIVB9vo6fyqBOzRu' 
access_token = '2362072416-DaoVm1luDLb9QGnl48MXaVXJXM73JpcbQAG75jx'  
access_token_secret = 'XOHk8Sc1Znb92Y0v9GZ3jPvtLsZA1IBKit9eODAsDNG8c'

In [24]:
import tweepy

#提交Key和secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#獲取類似於內容控制代碼的東西
api = tweepy.API(auth)

In [25]:
#定義如何抓取tweets
def get_tweets(listOfTweets, keyword, language, start, end):
    for tweet in tweepy.Cursor(api.search, q=keyword, lang = language, since = start, until = end, wait_on_rate_limit=True).items(10):
        if "RT" != tweet.text[0:2]:#刪除轉發
            articles = {'hastag':keyword,
                        'text': tweet.text.replace('\n', ''),
                        'time': str(tweet.created_at).split(" ")[0]
                       }
            listOfTweets.append(articles)   

    return listOfTweets

In [26]:
#設定抓取目標
hashtags = ['#Apple', '#Amazon', '#Facebook', '#Google', '#Microsoft']
hashtags

['#Apple', '#Amazon', '#Facebook', '#Google', '#Microsoft']

In [ ]:
#抓取tweets並存檔
import time

data0 = pd.DataFrame()
for hashtag in hashtags:
    print(hashtag)
    print("Start : %s" % time.ctime())
    
    corpus = []
    data = pd.DataFrame(get_tweets(listOfTweets = corpus, 
                                   keyword = hashtag, 
                                   language = 'en',
                                   start = start_date,
                                   end = end_date,
                                  ))
    print("End : %s" % time.ctime())
    #data.to_csv(hashtag + '_nonprocess_' + '.csv',index = False)
    data0 = pd.concat((data0, data), axis = 0)
    data0 = data0.reset_index(drop=True)
data0

In [117]:
data0 = pd.DataFrame()
for hashtag in hashtags:
    data = pd.read_csv(hashtag + '_nonprocess_' + '.csv')
    data0 = pd.concat((data0, data), axis = 0)
    data0 = data0.reset_index(drop=True)
data0

,hastag,text,time
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24
...,...,...,...
20273,#Facebook,"Hilarious, #Facebook says I have to ask Vivald...",2020-10-20
20274,#Facebook,I for one hope all the cash they’re pouring in...,2020-10-20
20275,#Facebook,Why are young people fed up in Nigeria? | #soc...,2020-10-20
20276,#Facebook,Project Veritas Drops A HUGE Inside Video For ...,2020-10-20


# feature_generation

## Preprocessed

In [118]:
import re
import copy
from nltk.tokenize import TweetTokenizer

data1 = copy.deepcopy(data0)

tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)#初始化tokenizer

data1["token"] = None

for i in range(len(data1)):
    #刪除url
    pattern1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')    # 匹配模式
    new_text1 = re.sub(pattern1, '', data1.iloc[i]["text"].lower())#先轉小寫
    #刪除hashtag
    pattern2 = re.compile(r'#(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')    # 匹配模式
    new_text2 = re.sub(pattern2, '', new_text1)
    #刪除...
    new_text3 = new_text2.rstrip('… ').rstrip('.')
    
    token = tknzr.tokenize(new_text3)
    
    data1.at[i, "token"] = token

data1.head()

,hastag,text,time,token
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho..."
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ..."
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features..."
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno..."
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ..."


## Sentiment Value

In [119]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sa = SentimentIntensityAnalyzer()

sentiment_value = []
for i in range(len(data1)):
    score = sa.polarity_scores(data1.iloc[i]["token"])#計算情緒分數
    sentiment_value.append(score['compound'])

data2 = pd.concat((data1, pd.DataFrame({'score' : sentiment_value})), axis = 1)
data2.head()

,hastag,text,time,token,score
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0


## Sentiment Proxy

In [143]:
sentiment = []
for i in data2['score']:
    if i >= 0:
        sentiment.append(1)#正面
    else:
        sentiment.append(0)#負面
data3 = pd.concat((data2, pd.DataFrame({'sentiment' : sentiment})), axis = 1)
data3.head()

,hastag,text,time,token,score,sentiment
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0,1
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0,1
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0,1
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0,1
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0,1


In [142]:
data3.sentiment.value_counts()

 0    19705
 1      438
-1      135
Name: sentiment, dtype: int64

## POS tag

In [144]:
#初始化詞性辨識器
import nltk
from collections import defaultdict
from nltk.corpus import treebank

#nltk.download('treebank')
POS_dict = defaultdict(dict)
for word_pos_pair in treebank.tagged_words():
    word = word_pos_pair[0].lower() 
    POS = word_pos_pair[1]
    POS_dict[word][POS] = POS_dict[word].get(POS,0) + 1

In [145]:
tagger_dict = {}
for word in POS_dict:
    tagger_dict[word] = max(POS_dict[word],key=lambda x: POS_dict[word][x])

def tags(sentence):
    return [(word,tagger_dict.get(word,"NN")) for word in sentence]

In [146]:
#辨識詞性
POS_tag = []
for i in range(len(data3)):
    if data3.iloc[i]['sentiment'] == 1:
        doc_POS_tag = []
        for tag in tags(data3.iloc[i]['token']):
            #是否有形容詞、副詞、動詞、名詞
            if tag[1] == 'JJ' or tag[1] == 'RB' or tag[1] == 'VB' or tag[1] == 'NN':
                doc_POS_tag.append(tag[0])
        POS_tag.append(doc_POS_tag)
    else:
        POS_tag.append(None)
data4 = pd.concat((data3, pd.DataFrame({'POS_tag' : POS_tag})), axis = 1)
data4.head()

,hastag,text,time,token,score,sentiment,POS_tag
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0,1,"[landscape, time, so]"
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0,1,"[help, learn]"
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0,1,"[microsoft, surface, pro, 2020, review]"
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0,1,"[rt, state, decay, official, announce]"
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0,1,"[udemy, free, discount, full, stack, science, ..."


## JJ_RB_VB_NN

In [147]:
JJ_RB_VB_NN = []
for i in data4['POS_tag']:
    if i == None:
        JJ_RB_VB_NN.append(0)
    else:
        JJ_RB_VB_NN.append(1)
data5 = pd.concat((data4, pd.DataFrame({'JJ_RB_VB_NN' : JJ_RB_VB_NN})), axis = 1)
data5.head()

,hastag,text,time,token,score,sentiment,POS_tag,JJ_RB_VB_NN
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0,1,"[landscape, time, so]",1
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0,1,"[help, learn]",1
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0,1,"[microsoft, surface, pro, 2020, review]",1
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0,1,"[rt, state, decay, official, announce]",1
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0,1,"[udemy, free, discount, full, stack, science, ...",1


## Antonym_value

In [202]:
#nltk.download('wordnet')
from nltk.corpus import wordnet 

antonyms_all = [] 

for i in range(len(data5)):
    if data5.iloc[i]['JJ_RB_VB_NN'] == 1:
        antonyms_doc = []
        for word in data5.iloc[i]['POS_tag']:
            for syn in wordnet.synsets(word): #該詞不同詞性下的所有涵意
                for l in syn.lemmas(): 
                    for i in l.antonyms():#反義詞
                        antonyms_doc.append(i.name())
        if antonyms_doc:                 
            antonyms_all.append(list(set(antonyms_doc)))
        else:
            antonyms_all.append(None)    
    else:
        antonyms_all.append(None)

data6 = pd.concat((data5, pd.DataFrame({'antonyms' : antonyms_all})), axis = 1)
data6.head()

,hastag,text,time,token,score,sentiment,POS_tag,JJ_RB_VB_NN,antonyms
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0,1,"[landscape, time, so]",1,None
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0,1,"[help, learn]",1,None
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0,1,"[microsoft, surface, pro, 2020, review]",1,"[overhead, amateur, subsurface, con, anti]"
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0,1,"[rt, state, decay, official, announce]",1,[unofficial]
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0,1,"[udemy, free, discount, full, stack, science, ...",1,"[lodge, thin, obstruct, incomplete, unnaturall..."


## flag

In [240]:
flag = []
for i in range(len(data6)):
    k = 0
    if data6['antonyms'][i]:#存在反義詞
        for j in data6['POS_tag'][i]:
            if j in data6['antonyms'][i]:#是否存在正反義詞對
                k = 1
                break
    if k == 1:
        flag.append(1)
    else:
        flag.append(0)
data7 = pd.concat((data6, pd.DataFrame({'flag' : flag})), axis = 1)
data7.head()

,hastag,text,time,token,score,sentiment,POS_tag,JJ_RB_VB_NN,antonyms,flag
0,#Microsoft,#StrategySaturdayThe #technology landscape cha...,2020-10-24,"[landscape, changes, all, the, time, ,, so, ho...",0.0,1,"[landscape, time, so]",1,None,0
1,#Microsoft,''#Microsoft partners #Netflix to help people ...,2020-10-24,"[', ', partners, to, help, people, learn, ai, ...",0.0,1,"[help, learn]",1,None,0
2,#Microsoft,Microsoft Surface Pro X 2020: Features and Rev...,2020-10-24,"[microsoft, surface, pro, x, 2020, :, features...",0.0,1,"[microsoft, surface, pro, 2020, review]",1,"[overhead, amateur, subsurface, con, anti]",0
3,#Microsoft,RT @DeadarticGames: State of Decay 3 - Officia...,2020-10-24,"[rt, :, state, of, decay, 3, -, official, anno...",0.0,1,"[rt, state, decay, official, announce]",1,[unofficial],0
4,#Microsoft,Udemy Free Discount - Full Stack Data Science ...,2020-10-24,"[udemy, free, discount, -, full, stack, data, ...",0.0,1,"[udemy, free, discount, full, stack, science, ...",1,"[lodge, thin, obstruct, incomplete, unnaturall...",0


## Groupby

In [241]:
company = data7.groupby("hastag")#分類hashtag
company_hashtag = list(company.size().index)
for hashtag in company_hashtag:#依序列出分各公司
    data = company.get_group(str(hashtag))#抓出特定公司的data
    print(hashtag)
    print(data.flag.value_counts())
    data.to_csv(str(hashtag) + '.csv', index = False)#儲存

#Amazon
0    265
1      1
Name: flag, dtype: int64
#Apple
0    4008
1      27
Name: flag, dtype: int64
#Facebook
0    6642
1      36
Name: flag, dtype: int64
#Google
0    8924
1      55
Name: flag, dtype: int64
#Microsoft
0    320
Name: flag, dtype: int64


## target

In [ ]:
target = []
for i in data7['label']:
    if i == '#sarcastic':
        target.append(1)
    else:
        target.append(0)
data8 = pd.concat((data7, pd.DataFrame({'target' : target})), axis = 1)
data8

## accurate

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(data8['target'], data8['flag']))
print(recall_score(data8['target'], data8['flag']))
print(f1_score(data8['target'], data8['flag']))

## feature
### the percentage of sarcastic tweets of daily stock tweets

In [8]:
sample = []
for i in range(0,5):
    data_feature = pd.read_csv(hashtags[i] + '.csv')
    data_feature.set_index("time", inplace=True)
    
    data_label = pd.read_csv(company_symbol[i] + '_stock_price' + '.csv')
    
    feature = []
    for stock_date in list(data_label['date']):
        try:#如果當天twitter上有關於那家公司的hashtag
            
            #先用stock_date的去搜尋出當天的所有tweets，再求sarcastic flag的數量
            feature_value_counts = data_feature.loc[stock_date]['flag'].value_counts()
            
            if feature_value_counts.size == 2:#如果有flag有0跟1
                feature.append(feature_value_counts[1] / feature_value_counts.sum())
            else:#如果flag只有0
                feature.append(0)
                
        except:#如果公司當天沒有被hashtag提及過
            feature.append(0)
        
    sample.append(feature)
    
sarcastic_feature = pd.DataFrame(sample).T
sarcastic_feature.columns = hashtags
sarcastic_feature

,#Apple,#Amazon,#Facebook,#Google,#Microsoft
0,0.000000,0.0,0.000000,0.000000,0.0
1,0.000000,0.0,0.000000,0.000000,0.0
2,0.009576,0.0,0.000000,0.000000,0.0
3,0.000000,0.0,0.000000,0.000000,0.0
4,0.010463,0.0,0.007194,0.006250,0.0
5,0.007722,0.0,0.005814,0.005561,0.0
6,0.003040,0.0,0.004103,0.007001,0.0


# Modeling

In [12]:
from xgboost.sklearn import XGBClassifier

pred = []
for i in range(0,5):
    X = pd.DataFrame(sarcastic_feature.iloc[:, i])
    Y = pd.read_csv(company_symbol[i] + '_stock_price' + '.csv')
    
    X_train = X.iloc[:-1]
    X_test = X.iloc[-1:]
    y_train = Y.iloc[1:, 2]#前一天的X預測今天的Y
    
    clf = XGBClassifier()
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    pred.append(y_pred[0])

pred

[-1, 0, -1, 1, 0]

#  Save

In [70]:
from datetime import datetime
last_date = datetime.strptime(Y.date.iloc[-1],  "%Y-%m-%d").date()#最後一天轉為時間型態

import datetime
pred_date = last_date + datetime.timedelta(days = 1)#時間加1變為預測日期

pred.insert(0, pred_date)#在預測數據前加上日期
data = pd.DataFrame(pred, index = ['date']+company_symbol).T
data.to_csv('prediction' + '.csv', index = False)#儲存 